In [ ]:
import re
sample_file_name = "../dutch-language/data/sample-sentences-nlwiki.txt"
short_file_name = "../dutch-language/data/short-sentences-nlwiki.txt"
full_file_name = "../dutch-language/data/clean-sentences-nlwiki.txt"
corrected_file_name = "../dutch-language/data/corrected-sentences-nlwiki.txt"

sentences_file_name = corrected_file_name
positives_file_name = "../dutch-language/data/positive-sentences-002.txt"

BATCH_SIZE = 500
TOTAL_SIZE = 10000000

regex_nd = re.compile(r'.*')


from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel(
    "roberta", "outputs/RoBERTa-004", use_cuda=True
)
print(type(model))


with open(sentences_file_name, 'r') as sentences_file:
    with open(positives_file_name, 'w') as positives_file:
        total_count = 0
        match_count = 0
        batch = []
        batch_count = len(batch)

        for line in sentences_file:
            # process lines
            clean_line = line.rstrip()
            if (regex_nd.match(clean_line)):
                match_count += 1

                # fill batch
                batch_count += 1
                batch.append(clean_line)
                if batch_count >= BATCH_SIZE:
                    # process batch
                    predictions, raw_outputs = model.predict(batch)
                    for index in range(len(predictions)):
                        is_positive = (predictions[index] == 1) # can be more subtle later; no, maybe, true
                        if (is_positive):
                            positives_file.write(f"1\t{raw_outputs[index][0]}\t{raw_outputs[index][1]}\t{batch[index]}\n")
                    batch = []
                    batch_count = len(batch)

            # total count
            total_count += 1
            if (total_count >= TOTAL_SIZE):
                break


print()
print(total_count)
print(match_count)

In [30]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel(
#    "roberta", "outputs/RoBERTa-006", use_cuda=True
    "bert", "outputs/BERT-002", use_cuda=True

)
print(type(model))


<class 'simpletransformers.classification.classification_model.ClassificationModel'>


In [32]:
predictions, raw_outputs = model.predict([
    "Ik word nieuwsgierig.",
    "Hoe word je gevraagd?",
    "Wat wordt je gevraagd?",
    "Wat word je opdringerig, zeg!",

    "Ik zend een pakje.",
    "Ik zend je een pakje.",
    "Ik zend u een heel mooie trui.",
    "Je zendt het toch nog vandaag op, hoop ik.", #
    "Jij zendt een hele grote doos naar oma.",
    "Zend je dat straks uit?", #
    "Zend jij echt die fiets via de post?",
    "Welke baas zendt je nu helemaal naar London voor 1 klant?",

    # With spelling mistake
    "Ik wordt nieuwsgierig.",
    "Hoe wordt je gevraagd?",
    "Wat word je gevraagd?",
    "Wat wordt je opdringerig, zeg!",

    "Ik zendt een pakje.",
    "Ik zendt je een pakje.",
    "Ik zendt u een heel mooie trui.", #
    "Je zend het toch nog vandaag op, hoop ik.",
    "Jij zend een hele grote doos naar oma.",
    "Zendt je dat straks uit?", #
    "Zendt jij echt die fiets via de post?",
    "Welke baas zend je nu helemaal naar London voor 1 klant?",

    # Trying other words
    # Correct spelling
    "Ik vind dit toch niet zo mooi.",
    "Wat vind jij van al die aandacht?",
    "Hoe vind ik nu de ingang?",
    "Het is toch erg dat hij dat niet vindt.",
    "Hoe zwaar vindt hij de opleiding?",

    "Ik loop er zo maar voorbij.",
    "Loop jij ook zo snel?",
    "Je loopt daar beter niet telkens over.",
    "Jij loopt echt helemaal naar zee?",
    "En daarom loopt hij er met een grote bocht omheen.",

    # With spelling mistake
    "Ik vindt dit toch niet zo mooi.",
    "Wat vindt jij van al die aandacht?",
    "Hoe vindt ik nu de ingang?",
    "Het is toch erg dat hij dat niet vind.", # Incorrectly evaluated to "0"
    "Hoe zwaar vind hij de opleiding?",

    "Ik loopt er zo maar voorbij.",
    "Loopt jij ook zo snel?",
    "Je loop daar beter niet telkens over.",
    "Jij loop echt helemaal naar zee?",
    "En daarom loop hij er met een grote bocht omheen."
])
print()
print(predictions)
print(raw_outputs)

# Expected           [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
# Actual RoBERTa-001 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1]
# Actual RoBERTa-002 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 1 1 1]
# Actual RoBERTa-003 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1]
# Actual    BERT-001 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0]



INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 6/6 [00:01<00:00,  4.19it/s]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1
 0 1 1 1 1 1 1]
[[ 5.480039  -5.5231433]
 [ 5.224986  -5.4824595]
 [ 5.391847  -5.5476246]
 [ 5.449721  -5.60282  ]
 [ 5.1760144 -5.5311527]
 [ 5.262928  -5.534521 ]
 [ 4.863189  -5.0584745]
 [ 5.346662  -5.6910825]
 [ 5.370072  -5.704945 ]
 [ 5.2930503 -5.7360206]
 [ 5.3439794 -5.693661 ]
 [ 5.238829  -5.612129 ]
 [-5.3600774  5.54054  ]
 [-5.0608287  5.33169  ]
 [-5.3027263  5.481593 ]
 [-5.2946205  5.415605 ]
 [-5.315498   5.537039 ]
 [-5.3030295  5.4397826]
 [-5.2565784  5.489816 ]
 [-5.2055383  5.110843 ]
 [-5.337029   5.358834 ]
 [-5.2701774  5.5503774]
 [-5.3040547  5.554106 ]
 [-5.319076   5.359827 ]
 [ 2.3163333 -2.5617633]
 [ 5.2101216 -5.570829 ]
 [ 5.2929597 -5.635382 ]
 [ 5.403548  -5.6927085]
 [ 5.4403987 -5.7646785]
 [ 4.9500036 -5.3146133]
 

In [ ]:
# Convert the strong positives (fp) to examples negatives
# cat positive-sentences-processed.txt | grep '^1.*\-[34]\.' | sed 's/.*\t\(..*\)$/    ["\1", 0],/' > strong-positives-to-correct.txt

In [33]:
# This is an experiment testing the algorithm on the Dutch language M.Sc. thesis of my daughter,
# one day before the due date ... Tested with newest RoBERTa and BERT based model, trained on
# exactly the same training (and validation) set:
#
# source data: 954 Dutch sentences written by my daughter about animation and psychology.
#
# RoBERTa: 0 positives ; I presume 0 or low amount of false negatives (low amount of actual positives in the source after manual proof reading)
# So this is "good", 0 false positives.

# BERT: 4 false positives; I presume 0 or low amount of false negatives (same reasoning)
# 
# The 4 false positives of BERT where
# 1	-4.757866382598877	4.64712381362915	In hoofdstuk 5 bekijk ik op welke manieren animatiefilm op dit moment al wordt geïmplementeerd in het therapeutische landschap.
# 1	-5.1004133224487305	5.195034503936768	Wel vond ik enkele case studies over de manieren waarop animatiefilm zoal geïmplementeerd wordt in de Re-animation benadering (Mason, 2009).
# 1	-4.971858501434326	5.104990005493164	Naast deze twee benaderingen vond ik nog een aantal andere projecten terug waarin animatiefilm wordt gemaakt binnen de therapeutische ruimte, maar waarover minder informatie terug te vinden is of waarvan het niet duidelijk is of ze op dit moment nog actief toegepast worden.
# 1	-1.145717978477478	0.7887341976165771	Wanneer ik de films sorteer op datum van release, wordt het duidelijk dat de films met een sterk en statistisch significant stereotiep beeld vooral de oudere Disney films betreffen.

import re
corrected_file_name = "../dutch-language/data/thesis-animatie_05_31-sentences-clean-end.txt"

sentences_file_name = corrected_file_name
positives_file_name = "../dutch-language/data/positive-sentences-animatie-001.txt"

BATCH_SIZE = 100
TOTAL_SIZE = 10000

regex_nd = re.compile(r'.*')


from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel(
    "bert", "outputs/BERT-002", use_cuda=True
# "roberta", "outputs/RoBERTa-006", use_cuda=True
)
print(type(model))


with open(sentences_file_name, 'r') as sentences_file:
    with open(positives_file_name, 'w') as positives_file:
        total_count = 0
        match_count = 0

        batch = []
        batch_count = len(batch)

        for line in sentences_file:
            # process lines
            clean_line = line.rstrip()
            if (regex_nd.match(clean_line)):
                match_count += 1

                # fill batch
                batch_count += 1
                batch.append(clean_line)
                if batch_count >= BATCH_SIZE:
                    # process batch
                    predictions, raw_outputs = model.predict(batch)
                    for index in range(len(predictions)):
                        is_positive = (predictions[index] == 1) # can be more subtle later; no, maybe, true
                        if (is_positive):
                            positives_file.write(f"1\t{raw_outputs[index][0]}\t{raw_outputs[index][1]}\t{batch[index]}\n")
                    batch = []
                    batch_count = len(batch)

            # total count
            total_count += 1
            if (total_count >= TOTAL_SIZE):
                break


print()
print(total_count)
print(match_count)

<class 'simpletransformers.classification.classification_model.ClassificationModel'>
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 13/13 [00:03<00:00,  4.32it/s]
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 13/13 [00:03<00:00,  4.22it/s]
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 13/13 [00:02<00:00,  4.41it/s]
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 13/13 [00:03<00:00,  4.06it/s]
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 13/13 [00:03<00:00,  4.10it/s]
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100

In [ ]:
*   